# Adding vehicles to plans

In this example, we show how an existing set of plans can be updated to include additional vehicles.

In [1]:
import logging
import os
from pprint import pprint
import random
from pathlib import Path

import pam
from pam import vehicles
from pam import read, write

In [2]:
# load up example population
data_path = Path("data/example_data")
pop = read.read_matsim(data_path / "example_plans.xml")
pop.stats

{'num_households': 51,
 'num_people': 51,
 'num_activities': 153,
 'num_legs': 102}

## Simple example of vehicle assignment

Below we assign vehicles to agents for the "car" mode.

In [3]:
# first define some vehicle types and add them to the population via the vehicles_manager
pop.vehicles_manager.add_type("small_car", vehicles.VehicleType())
pop.vehicles_manager.add_type("large_car", vehicles.VehicleType(length=15, width=2))

In [4]:
# now we randomly assign some vehicles and electric vehicles using these types

for hid, pid, p in pop.people():
    if random.random() < 0.2:  # 20% change of having an ev
        # evs have the type "small_car"
        p.vehicles = {"car": vehicles.ElectricVehicle(pid, type_id="small_car", battery_capacity=100)}
    else:
        if random.random() < 0.5:  # 40% change of having a regular vehicle of type "small_car"
            p.vehicles = {"car": vehicles.Vehicle(pid, type_id="small_car")}
        else:  # 40% change of having a vehicle of type "ev"
            p.vehicles = {"car": vehicles.Vehicle(pid, type_id="large_car")}
   
    # note that we specify the key as "car" as this is the transport "mode"

So far we are using a lot of default values (especially for the ElectricVehciles). Defaults follow the same defaults as defined in MATSim's dtd files; [vehicleDefinitions_v2.0.xsd](https://www.matsim.org/files/dtd/vehicleDefinitions_v2.0.xsd) and [electric_vehicles_v1.dtd](https://www.matsim.org/files/dtd/electric_vehicles_v1.dtd).

We can use the standard pam.write to write these vehicle formats to disk by providing a vehicles and, optionally, an electric vehciles path.

In [5]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

write.write_matsim(
    pop,
    plans_path="./tmp/plans.xml",
    vehs_path="./tmp/vehicles.xml",
    evs_path="./tmp/evs.xml"
    )

INFO:root:Building population vehicles output.
INFO:root:Writing vehicle types to ./tmp/vehicles.xml
INFO:root:Writing vehicles to ./tmp/vehicles.xml
INFO:root:Writing electric vehicles to ./tmp/evs.xml


## Modifying vehicles in existing MATSim population

Pam can also read vehicles (and electric vehicles) from an existing MATSim population. These can then be checked and/or modified before writing a new population.

In [6]:
pop = read.read_matsim(
    plans_path = data_path / "plans.xml",
    all_vehicles_path = data_path / "vehicles.xml",
    electric_vehicles_path = data_path / "evs.xml"
    )

In [7]:
pprint(pop.vehicles_manager.veh_types)
pprint(pop.random_person().vehicles)
print(f"Population has {len([v for _, _, _, v  in pop.vehicles() if v.type_id == 'small_car'])} small cars.")
print(f"Population has {len(list(pop.evs()))} evs.")

{'large_car': VehicleType(length=15.0, width=2.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0),
 'small_car': VehicleType(length=7.5, width=1.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0)}
{'car': Vehicle(vid='census_49', type_id='large_car')}
Population has 31 small cars.
Population has 11 evs.


In [8]:
# 50% chance of large_car type switching to small_car
for _, pid, person in pop.people():
    veh = person.vehicles.get("car")
    if isinstance(veh, vehicles.Vehicle) and veh.type_id == "large_car":
        if random.random() < .5:
            person.vehicles["car"] = vehicles.Vehicle(pid, "small_car")

In [9]:
pprint(pop.vehicles_manager.veh_types)
pprint(pop.random_person().vehicles)
print(f"Population has {len([v for _, _, _, v  in pop.vehicles() if v.type_id == 'small_car'])} small cars.")
print(f"Population has {len(list(pop.evs()))} evs.")

{'large_car': VehicleType(length=15.0, width=2.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0),
 'small_car': VehicleType(length=7.5, width=1.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0)}
{'car': Vehicle(vid='census_5', type_id='large_car')}
Population has 41 small cars.
Population has 11 evs.


In [10]:
# 10% chance of small_car type switching to electric vehcile
for _, pid, person in pop.people():
    veh = person.vehicles.get("car")
    if isinstance(veh, vehicles.Vehicle) and veh.type_id == "small_car":
        if random.random() < .5:
            person.vehicles["car"] = vehicles.ElectricVehicle(pid, "small_car")

In [11]:
pprint(pop.vehicles_manager.veh_types)
pprint(pop.random_person().vehicles)
print(f"Population has {len([v for _, _, _, v  in pop.vehicles() if v.type_id == 'small_car'])} small cars.")
print(f"Population has {len(list(pop.evs()))} evs.")

{'large_car': VehicleType(length=15.0, width=2.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0),
 'small_car': VehicleType(length=7.5, width=1.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0)}
{'car': Vehicle(vid='census_46', type_id='large_car')}
Population has 41 small cars.
Population has 28 evs.


In [12]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

write.write_matsim(
    pop,
    plans_path="./tmp/plans.xml",
    vehs_path="./tmp/vehicles.xml",
    evs_path="./tmp/evs.xml"
    )

INFO:root:Building population vehicles output.
INFO:root:Writing vehicle types to ./tmp/vehicles.xml
INFO:root:Writing vehicles to ./tmp/vehicles.xml
INFO:root:Writing electric vehicles to ./tmp/evs.xml
